# Adaptive_Quiz Class Documentation

The `Adaptive_Quiz` class is designed to generate and manage an adaptive quiz that adjusts the difficulty of questions based on the user's responses. This allows for a more personalized learning experience.

## Class Overview

### Attributes

- **`custom_template`**: A template string used to generate a set number of multiple-choice questions (MCQs) based on the provided topic, learning objective, and difficulty level.
  
- **`adaptive_template`**: A template string used to generate new questions based on the user's response to the previous question. The difficulty of the questions adapts depending on whether the user's response was correct or incorrect.

### Constructor (`__init__`)

The constructor initializes the `Adaptive_Quiz` object with the following parameters:

- **`db`** (`str`): Specifies the database to be used. Default is `None`. If `"supabase"` is provided, a connection to Supabase will be initialized.
  
- **`llm`** (`object`): The language model (LLM) used for generating questions. If not provided, the class will automatically initialize an LLM.
  
- **`difficulty_increase_threshold`** (`str`): The threshold for increasing the difficulty of questions. Default is `"Medium"`.
  
- **`topic`** (`str`): The topic of the quiz. Default is an empty string (`""`).
  
- **`num_questions`** (`int`): The number of questions to generate in the quiz. Default is `5`.
  
- **`custom_instruction`** (`str`): Custom instructions for generating the quiz questions. Default is an empty string (`""`).
  
- **`show_options`** (`bool`): A boolean indicating whether to immediately show answer options to the user. Default is `False`.
  
- **`data`** (`any`): External data that might influence question generation. Default is `None`.
  
- **`source_type`** (`str`): Specifies the source type for quiz data. Default is `None`.

### Internal Attributes

- **`quiz_data`** (`list`): A list to store the quiz data.
  
- **`start_time`** (`datetime`): Tracks the start time of the quiz session.
  
- **`supabase`** (`object`): A connection object to Supabase, initialized if `db` is set to `"supabase"`.

### Methods

#### `initialize_llm()`
- **Description**: Initializes the language model (LLM) to be used for generating quiz questions.

#### `initialize_supabase()`
- **Description**: Initializes a Supabase connection if the `db` parameter is set to `"supabase"`.

## Example Usage

```python
quiz = Adaptive_Quiz(
    difficulty_increase_threshold="High", #easy/medium/high
    topic="Mathematics",
    num_questions=10,
    custom_instruction="Focus on calculus.",
    show_options=True,
    data=,"path/to/data" #pdf/url/text
    source_type="URL" #pdf/url/text
)


# Installing Dependencies

In [1]:
!pip install educhain langchain_openai supabase --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318

#Import Modules

In [2]:
import os
import time
from educhain import qna_engine
from langchain_openai import ChatOpenAI
from supabase import create_client, Client
from google.colab import userdata

## Class Initialization

In [3]:
class Adaptive_Quiz:

    custom_template = """
    Generate {num} multiple-choice question (MCQ) based on the given topic and level.
    Provide the question, four answer options, and the correct answer.

    Topic: {topic}
    Learning Objective: {learning_objective}
    Difficulty Level: {difficulty_level}
    """

    adaptive_template = """
    Based on the user's response to the previous question on {topic}, generate a new multiple-choice question (MCQ).
    If the user's response is correct, output a harder question. Otherwise, output an easier question.
    Provide the question, four answer options, and the correct answer.

    Previous Question: {previous_question}
    User's Response: {user_response}
    Was the response correct?: {response_correct}
    """

    def __init__(self, db=None, llm=None, difficulty_increase_threshold="Medium", topic="", num_questions=5, custom_instruction="", show_options=False, data=None, source_type=None):
        self.db = db
        self.llm = llm or self.initialize_llm()
        self.difficulty_increase_threshold = difficulty_increase_threshold
        self.topic = topic
        self.num_questions = num_questions
        self.custom_instruction = custom_instruction
        self.quiz_data = []
        self.start_time = None
        self.show_options = show_options
        self.data = data
        self.source_type = source_type

        self.supabase = None
        if db == "supabase":
            self.supabase = self.initialize_supabase()

    @staticmethod
    def initialize_llm():
        api_key = userdata.get("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI Key not found in environment variables.")
        return ChatOpenAI(
            model="gpt-4o-mini",
            #openai_api_base="https://api.groq.com/openai/v1",
            openai_api_key=api_key
        )

    @staticmethod
    def initialize_supabase():
        url = os.getenv("SUPABASE_URL")
        key = os.getenv("SUPABASE_KEY")
        if not url or not key:
            raise ValueError("Supabase URL or Key not found in environment variables.")
        return create_client(url, key)

    def generate_initial_question(self):
        if self.data:
            result = qna_engine.generate_mcqs_from_data(
                source=self.data,
                source_type=self.source_type,
                num=1,
                llm=self.llm,
            )
        else:
            result = qna_engine.generate_mcq(
                topic=self.topic,
                num=1,
                learning_objective=f"General knowledge of {self.topic}",
                difficulty_level=self.difficulty_increase_threshold,
                llm=self.llm,
                prompt_template=self.custom_template,  # Use self.custom_template
            )
        return result.questions[0] if result and result.questions else None

    def generate_next_question(self, previous_question, user_response, response_correct):
        if self.data:
            result = qna_engine.generate_mcqs_from_data(
                source=self.data,
                source_type=self.source_type,
                num=1,
                llm=self.llm,
            )
        else:
            result = qna_engine.generate_mcq(
                topic=self.topic,
                num=1,
                llm=self.llm,
                prompt_template=self.adaptive_template,  # Use self.adaptive_template
                previous_question=previous_question,
                user_response=user_response,
                response_correct=response_correct
            )
        return result.questions[0] if result and result.questions else None

    def start_quiz(self):
        self.start_time = time.time()
        question_number = 0
        score = 0

        current_question = self.generate_initial_question()
        while question_number < self.num_questions and current_question:
            print(f"Question {question_number + 1}: {current_question.question}")
            if self.show_options:
                for i, option in enumerate(current_question.options):
                    print(f"{i+1}. {option}")
                user_answer = input("Select the correct option number: ")
                user_answer = current_question.options[int(user_answer) - 1]
            else:
                user_answer = input("Your answer: ")
            correct_answer = current_question.answer

            if user_answer == correct_answer:
                print("Correct!")
                score += 1
                response_correct = "True"
            else:
                print(f"Incorrect. The correct answer was {correct_answer}.")
                response_correct = "False"

            # Log quiz data
            self.quiz_data.append({
                "question_number": question_number + 1,
                "question": current_question.question,
                "user_answer": user_answer,
                "correct_answer": correct_answer,
                "response_correct": response_correct,
            })

            # Generate the next question
            question_number += 1
            current_question = self.generate_next_question(
                current_question.question,
                user_answer,
                response_correct
            )

        total_time = time.time() - self.start_time
        print(f"Quiz completed! Final Score: {score}/{self.num_questions}. Total Time: {total_time:.2f} seconds")

        if self.supabase:
            self.save_to_supabase(score, total_time)

    def save_to_supabase(self, score, total_time):
        try:
            data = {
                "topic": self.topic,
                "difficulty_increase_threshold": self.difficulty_increase_threshold,
                "num_questions": self.num_questions,
                "score": score,
                "total_time": total_time,
                "quiz_data": self.quiz_data
            }
            print(data)
            response = self.supabase.table("quiz_results").insert(data).execute()
            if response.status_code != 201:
                raise Exception(f"Failed to save quiz data to Supabase. Response: {response.data}")
            print("Quiz data successfully saved to Supabase.")
        except Exception as e:
            print(f"An error occurred while saving to Supabase: {e}")




In [4]:
# Example Use Case
if __name__ == "__main__":
    quiz = Adaptive_Quiz(
        topic="Python Programming",
        num_questions=3
    )
    quiz.start_quiz()

Question 1: What is the output of the following code: print(type([]) is list)?
Your answer: list
Incorrect. The correct answer was True.
Question 2: What will the following code output? print(5 // 2)
Your answer: 2
Correct!
Question 3: What will the following code output? print(3 ** 2)
Your answer: 9
Correct!
Quiz completed! Final Score: 2/3. Total Time: 30.61 seconds


## Added Data support as well

In [5]:
quiz = Adaptive_Quiz(
    topic="Python",
    num_questions=1,
    show_options=True,
    data="https://en.wikipedia.org/wiki/Python_(programming_language)",
    source_type="url"
)
quiz.start_quiz()

Question 1: What is the primary design philosophy of Python?
1. Code complexity over simplicity
2. Code readability and simplicity
3. Using punctuation over whitespace for structure
4. Strictly enforcing static typing
Select the correct option number: 3
Incorrect. The correct answer was Code readability and simplicity.
Quiz completed! Final Score: 0/1. Total Time: 12.37 seconds


## Routine Testing

In [ ]:
if __name__ == "__main__":
    quiz = Adaptive_Quiz(

        topic="Python Programming",
        num_questions=1

    )
    quiz.start_quiz()

Question 1: What is the output of the following code: print(type([]) is list)?
Your answer: list
Incorrect. The correct answer was True.
Quiz completed! Final Score: 0/1. Total Time: 8.80 seconds


  ## Accessing native functions

In [ ]:
quiz = Adaptive_Quiz(
    topic="Python",
    num_questions=5,
    show_options=True,
)

question = quiz.generate_initial_question()
print(question.question)
print(question.options)
print(question.answer)

What is the output of the following Python code: print(type([]) is list)?
['True', 'False', 'None', 'TypeError']
True


In [ ]:
question.show()

Question: What is the output of the following Python code: print(type([]) is list)?
Options:
  A. True
  B. False
  C. None
  D. TypeError

Correct Answer: True
Explanation: In Python, the 'type' function returns the type of an object. Since '[]' is an empty list, 'type([])' returns 'list', and comparing it with 'list' using 'is' will return True.



In [ ]:
!curl --request GET \
     --url https://api.play.ht/api/v2/voices \
     --header 'accept: application/json'

{"error_message":"An authorization header must be provided. Please refer to https://docs.play.ht/reference/api-authentication for more info.","error_id":"UNAUTHORIZED"}

## Supabase Integration Testing

In [ ]:
%env SUPABASE_URL = "URL"

%env SUPABASE_KEY = "KEY"

env: SUPABASE_URL="URL"
env: SUPABASE_KEY="KEY"


In [ ]:
if __name__ == "__main__":
    quiz = Adaptive_Quiz(
        db = 'supabase',
        topic="Python Programming",
        num_questions=1

    )
    quiz.start_quiz()

Question 1: What is the output of the following Python code: print(type([]))?
Your answer: list
Correct!
Quiz completed! Final Score: 1/1. Total Time: 6.05 seconds
{'topic': 'Python Programming', 'difficulty_increase_threshold': 'Medium', 'num_questions': 1, 'score': 1, 'total_time': 6.051225423812866, 'quiz_data': [{'question_number': 1, 'question': 'What is the output of the following Python code: print(type([]))?', 'user_answer': 'list', 'correct_answer': 'list', 'response_correct': 'True'}]}
An error occurred while saving to Supabase: {}


## SQL SCHEMA Creation

In [ ]:
import os
import time
import json
from educhain import qna_engine
from langchain_openai import ChatOpenAI
from supabase import create_client, Client

class Adaptive_Quiz:
    custom_template = """
    Generate {num} multiple-choice question (MCQ) based on the given topic and level.
    Provide the question, four answer options, and the correct answer.

    Topic: {topic}
    Learning Objective: {learning_objective}
    Difficulty Level: {difficulty_level}
    """

    adaptive_template = """
    Based on the user's response to the previous question on {topic}, generate a new multiple-choice question (MCQ).
    If the user's response is correct, output a harder question. Otherwise, output an easier question.
    Provide the question, four answer options, and the correct answer.

    Previous Question: {previous_question}
    User's Response: {user_response}
    Was the response correct?: {response_correct}
    """

    def __init__(self, db=None, llm=None, difficulty_increase_threshold="Medium", topic="", num_questions=5, custom_instruction=""):
        self.db = db
        self.llm = llm or self.initialize_llm()
        self.difficulty_increase_threshold = difficulty_increase_threshold
        self.topic = topic
        self.num_questions = num_questions
        self.custom_instruction = custom_instruction
        self.quiz_data = []
        self.start_time = None

        self.supabase = None
        if db == "supabase":
            self.supabase = self.initialize_supabase()
            self.create_table_if_not_exists()

    @staticmethod
    def initialize_llm():
        api_key = userdata.get("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("GROQ API Key not found in environment variables.")
        return ChatOpenAI(
            model="gpt-4o-mini",
            openai_api_base="https://api.groq.com/openai/v1",
            openai_api_key=api_key
        )

    @staticmethod
    def initialize_supabase():
        url = os.getenv("SUPABASE_URL")
        key = os.getenv("SUPABASE_KEY")
        if not url or not key:
            raise ValueError("Supabase URL or Key not found in environment variables.")
        return create_client(url, key)

    def create_table_if_not_exists(self):
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS quiz_results (
            id SERIAL PRIMARY KEY,
            topic TEXT NOT NULL,
            difficulty_increase_threshold TEXT NOT NULL,
            num_questions INTEGER NOT NULL,
            score INTEGER NOT NULL,
            total_time FLOAT NOT NULL,
            quiz_data JSONB NOT NULL,
            created_at TIMESTAMPTZ DEFAULT CURRENT_TIMESTAMP
        );
        """
        try:
            response = self.supabase.rpc("execute_sql", {"query": create_table_sql}).execute()
            print("Table creation response:", response)
        except Exception as e:
            print(f"An error occurred while creating the table: {e}")

    def generate_initial_question(self):
        result = qna_engine.generate_mcq(
            topic=self.topic,
            num=1,
            learning_objective=f"General knowledge of {self.topic}",
            difficulty_level=self.difficulty_increase_threshold,
            llm=self.llm,
            prompt_template=self.custom_template,
        )
        return result.questions[0] if result and result.questions else None

    def generate_next_question(self, previous_question, user_response, response_correct):
        result = qna_engine.generate_mcq(
            topic=self.topic,
            num=1,
            llm=self.llm,
            prompt_template=self.adaptive_template,
            previous_question=previous_question,
            user_response=user_response,
            response_correct=response_correct
        )
        return result.questions[0] if result and result.questions else None

    def start_quiz(self):
        self.start_time = time.time()
        question_number = 0
        score = 0

        current_question = self.generate_initial_question()
        while question_number < self.num_questions and current_question:
            print(f"Question {question_number + 1}: {current_question.question}")
            user_answer = input("Your answer: ")
            correct_answer = current_question.answer

            if user_answer == correct_answer:
                print("Correct!")
                score += 1
                response_correct = "True"
            else:
                print(f"Incorrect. The correct answer was {correct_answer}.")
                response_correct = "False"

            # Log quiz data
            self.quiz_data.append({
                "question_number": question_number + 1,
                "question": current_question.question,
                "user_answer": user_answer,
                "correct_answer": correct_answer,
                "response_correct": response_correct,
            })

            # Generate the next question
            question_number += 1
            current_question = self.generate_next_question(
                current_question.question,
                user_answer,
                response_correct
            )

        total_time = time.time() - self.start_time
        print(f"Quiz completed! Final Score: {score}/{self.num_questions}. Total Time: {total_time:.2f} seconds")

        if self.supabase:
            self.save_to_supabase(score, total_time)

    def save_to_supabase(self, score, total_time):
        try:
            data = {
                "topic": self.topic,
                "difficulty_increase_threshold": self.difficulty_increase_threshold,
                "num_questions": self.num_questions,
                "score": score,
                "total_time": total_time,
                "quiz_data": json.dumps(self.quiz_data)  # Ensure quiz_data is JSON-encoded
            }

            # Print the data being sent to Supabase for debugging purposes
            print("Saving the following data to Supabase:", data)

            response = self.supabase.table("quiz_results").insert(data).execute()

            # Print the response from Supabase for debugging purposes
            print("Supabase response:", response)

            if response.status_code != 201:
                raise Exception(f"Failed to save quiz data to Supabase. Response: {response.data}")
            print("Quiz data successfully saved to Supabase.")
        except Exception as e:
            print(f"An error occurred while saving to Supabase: {e}")

# Example Use Case
if __name__ == "__main__":
    quiz = Adaptive_Quiz(
        db="supabase",
        topic="Python Programming",
        num_questions=1
    )
    quiz.start_quiz()


An error occurred while creating the table: {'code': 'PGRST202', 'details': 'Searched for the function public.execute_sql with parameter query or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.', 'hint': None, 'message': 'Could not find the function public.execute_sql(query) in the schema cache'}


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}